# Multiple convolution layers

In [1]:
# import convolution layers in 2-Dimensional image
from keras.layers import Conv2D

# Use a Flatten layer to be like a bridge between a convolution layer and a dense layer
from keras.layers import Flatten

# Other modules are the same
from keras.layers import Dense
from keras.models import Sequential    # 仍然使用Sequential model
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [3]:
model = Sequential()

# first cnn layer
model.add(Conv2D(10, kernel_size = 2, activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))

# second cnn layer
model.add(Conv2D(10, kernel_size = 2, activation = 'relu'))

# Flatten layer
model.add(Flatten())

# Output layer
model.add(Dense(3, activation = 'softmax'))

# Why adding multiple CNN layers?

An example of multiple CNN
![](Image/Image10.jpg)

1. Layers in the early part of the network tend to respond to oriented lines or **simple textures**.
2. Intermediate layers of the network tend to respond to **more complex** features including simple objects, such as eyes.
3. For higher layers of the network, the feature maps tend to extract **specific types of objects**, such as dogs, cats and cars.

Therefore, we can build up objects in an image from simple features to more complex features and distinct categories of objects

However, a very deep CNN requires more time and data to train

---

# Consideration of the number of parameters

## Comparing dense network and convolution network

### A fully connected network (Dense)

In [7]:
model = Sequential()

# first cnn layer
model.add(Dense(10, activation = 'relu', input_shape = (784, )))

# second cnn layer
model.add(Dense(10, activation = 'relu'))

# Output layer
model.add(Dense(3, activation = 'softmax'))

In [8]:
# summarise the model to check number of parameters
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total params: 7,993
Trainable params: 7,993
Non-trainable params: 0
_________________________________________________________________


第一層：784個pixels * 10個neurons + 10個bias units = 7850 <br/>
第二層：第一層的10個neurons * 10個neurons + 10個bias units = 110 <br/>
第三層：第二層的10個neurons * 3個neurons + 3個bias units = 33

### A convolutional network

In [13]:
model = Sequential()

# first cnn layer
model.add(Conv2D(10, kernel_size = 3, activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))

# second cnn layer
model.add(Conv2D(10, kernel_size = 3, activation = 'relu', padding = 'same'))

# Flatten layer
model.add(Flatten())

# Output layer
model.add(Dense(3, activation = 'softmax'))

In [14]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 10)        100       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 10)        910       
_________________________________________________________________
flatten_4 (Flatten)          (None, 7840)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 23523     
Total params: 24,533
Trainable params: 24,533
Non-trainable params: 0
_________________________________________________________________


第一層：10個kernels * Kernel_Size為(3 * 3) + 10個bias units = 100 <br/>
第二層：前一層10個kernels * 這一層10個kernels * 每個kernel的總pixels為(3 * 3) + 10個bias units = 910 <br/>
第三層：Flatten沒有參數，只會將第二層的10個feature map合併成一個Array <br/>
第四層：由於第一層與第二層都有zero padding，因此出來的每一個feature map的size都跟原來的一樣(784 pixels)，而第二層傳出10個feature maps，因此有784 * 10個變量。然後第四層有3個neurons，因此第四層的參數總數為 $784 * 10 * 3 + 3個bias = 23523$

---

## Another example

### A dense network

In [15]:
model = Sequential()

# first cnn layer
model.add(Dense(5, activation = 'relu', input_shape = (784, )))

# second cnn layer
model.add(Dense(15, activation = 'relu'))

# Output layer
model.add(Dense(3, activation = 'softmax'))

In [16]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 5)                 3925      
_________________________________________________________________
dense_9 (Dense)              (None, 15)                90        
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 48        
Total params: 4,063
Trainable params: 4,063
Non-trainable params: 0
_________________________________________________________________


### A convolutional network

In [17]:
model = Sequential()

# first cnn layer
model.add(Conv2D(5, kernel_size = 3, activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))

# second cnn layer
model.add(Conv2D(15, kernel_size = 3, activation = 'relu', padding = 'same'))

# Flatten layer
model.add(Flatten())

# Output layer
model.add(Dense(3, activation = 'softmax'))

In [18]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 5)         50        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 15)        690       
_________________________________________________________________
flatten_5 (Flatten)          (None, 11760)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 35283     
Total params: 36,023
Trainable params: 36,023
Non-trainable params: 0
_________________________________________________________________


## Summary

A dense network tends to have more parameters at the first layer, while a convolutional network tends to have more parameters at the output layer

---

# Pooling operations

A large number of parameters of the convolutional network can be a challenge. Summarizing the output of convolutional layers in concise manner is a way to mitigate this problem, and it's called **"Pooling"** (是一種資訊壓縮的概念)

1.  Max Pooling: <br/>
假設有一張28 * 28的image，今天我用一個2 * 2的window來壓縮。一開始，該window位於image的最左上角，擷取到最左上角2 * 2這塊區域的4個pixels的值。然後將整塊區域用這4個pixels中最大的值取代，使左上角這四個pixels的數值都是原本四個的最大值，等於四個合而為一，並保留最大值。一直重複相同的步驟直到window掃完整張image，這時資料的大小會變成原本的$1/4$

### Step-by-step Implementation

In [24]:
# Allocating the output (consider the size of the window is 2*2)
im = np.array([[1,0,1,1], 
              [1,0,0,1], 
              [0,0,1,1], 
              [1,0,0,0]])

result = np.zeros((int(im.shape[0]/2), int(im.shape[1]/2)))

# First, 左上角第一個對應image左上角四個
result[0, 0] = np.max(im[0:2, 0:2])

# Second, window向右移動兩個pixels(不重複計算)
result[0, 1] = np.max(im[0:2, 2:4])

# Keep moving our window
result[0, 2] = np.max(im[0:2, 4:6])
# Until we have done the first two rows of the image

# Then start over at the second row of the result
result[1, 0] = np.max(im[2:4, 0:2])
result[1, 1] = np.max(im[2:4, 2:4])
result[1, 2] = np.max(im[2:4, 4:6])

# Until we have gone through every pixel of the image

(4, 4)


ValueError: zero-size array to reduction operation maximum which has no identity

### Loop Implementation

In [27]:
im = np.array([[1,0,1,1], 
              [1,0,0,1], 
              [0,0,1,1], 
              [0,0,0,0]])

result = np.zeros((int(im.shape[0]/2), int(im.shape[1]/2)))
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        result[i, j] = np.max(im[2*i:2*i+2, 2*j:2*j+2])
        
result

array([[1., 1.],
       [0., 1.]])

## Max Pooling in Keras

In [28]:
# Import MaxPool2D for max pooling
from keras.layers import MaxPool2D

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

In [30]:
# Build our network just like before
model = Sequential()

model.add(Conv2D(5, kernel_size = 3, activation = 'relu', input_shape = (28, 28, 1)))

# After each convolutional layer, we add a POOLING layer (argument 2 means window size = 2)
model.add(MaxPool2D(2))

# second convolutional layer
model.add(Conv2D(15, kernel_size = 3, activation = 'relu', padding = 'same'))

# After each convolutional layer, we add a POOLING layer (argument 2 means window size = 2)
model.add(MaxPool2D(2))

# The rest of layers are the same
model.add(Flatten())
model.add(Dense(3, activation = 'softmax'))

In [31]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 5)         50        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 5)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 15)        690       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 15)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 540)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 1623      
Total params: 2,363
Trainable params: 2,363
Non-trainable params: 0
____________________________________________________

**According to the result, the number of parameters of the model dramatically decreases.**